In [1]:
import os
import torch
import accelerate

from rd3d.datasets import build_dataloader
from rd3d.models import build_detector
from rd3d.core import Config
from rd3d import PROJECT_ROOT

os.chdir(PROJECT_ROOT)
acc = accelerate.Accelerator()
cfg = Config.fromfile_py("configs/voxformer/voxformer_4x2_80e_kitti_3cls.py")
dataloader = build_dataloader(cfg.DATASET, cfg.RUN)
model = build_detector(cfg.MODEL, dataset=dataloader.dataset).cuda()
batch_dict = next(iter(dataloader))
dataloader.dataset.load_data_to_gpu(batch_dict)

[2023-12-17 14:44:40,284 cfg INFO] import module at root: /home/nrsl/workspace/temp/voxformer
[2023-12-17 14:44:40,284 cfg INFO] import module as config: configs.voxformer.voxformer_4x2_80e_kitti_3cls
[2023-12-17 14:44:40,404 dataset INFO] Database filter by min points Car: 14357 => 13532
[2023-12-17 14:44:40,405 dataset INFO] Database filter by min points Pedestrian: 2207 => 2168
[2023-12-17 14:44:40,406 dataset INFO] Database filter by min points Cyclist: 734 => 705
[2023-12-17 14:44:40,426 dataset INFO] Database filter by difficulty Car: 13532 => 10759
[2023-12-17 14:44:40,431 dataset INFO] Database filter by difficulty Pedestrian: 2168 => 2075
[2023-12-17 14:44:40,433 dataset INFO] Database filter by difficulty Cyclist: 705 => 581
[2023-12-17 14:44:40,440 dataset INFO] Loading KITTI dataset
[2023-12-17 14:44:40,559 dataset INFO] Total samples for KITTI dataset: 3712


In [13]:
# vfe = model.vfe
# bb3d = model.backbone_3d
# batch_dict = vfe(batch_dict)
# batch_dict = bb3d(batch_dict)
batch_dict = model.vfe(batch_dict)
vox_numbs = batch_dict['voxel_numbers']
vox_coors = batch_dict['voxel_coords']
vox_feats = batch_dict['voxel_features']
ind, vox_nums = model.backbone_3d.mapping(vox_numbs, vox_coors)
ind1, ind2, ind12, ind21 = ind
vox_feats = vox_feats[ind1].view(-1, model.backbone_3d.group_size, vox_feats.size(-1))

linear = torch.nn.Linear(vox_feats.size(-1), 10).cuda()
bn = torch.nn.BatchNorm1d(10)
vox_feats = linear(vox_feats)
print(vox_feats.shape)
# vox_feats = bn(vox_feats)


torch.Size([1727, 32, 10])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:17                                                                                   │
│                                                                                                  │
│   14 bn = torch.nn.BatchNorm1d(10)                                                               │
│   15 vox_feats = linear(vox_feats)                                                               │
│   16 print(vox_feats.shape)                                                                      │
│ ❱ 17 vox_feats = bn(vox_feats)                                                                   │
│   18                                                                                             │
│                                                                                                  │
│ /home/nrsl/software/anaconda3/envs/torch112/lib/python3.7/site-packages/torch/nn/modules/module. │
│ py:1130 in _call_impl                                                                            │
│                                                                                                  │
│   1127 │   │   # this function, and just call forward.                                           │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1131 │   │   # Do not call functions when jit is used                                          │
│   1132 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1133 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /home/nrsl/software/anaconda3/envs/torch112/lib/python3.7/site-packages/torch/nn/modules/batchno │
│ rm.py:179 in forward                                                                             │
│                                                                                                  │
│   176 │   │   │   self.bias,                                                                     │
│   177 │   │   │   bn_training,                                                                   │
│   178 │   │   │   exponential_average_factor,                                                    │
│ ❱ 179 │   │   │   self.eps,                                                                      │
│   180 │   │   )                                                                                  │
│   181                                                                                            │
│   182                                                                                            │
│                                                                                                  │
│ /home/nrsl/software/anaconda3/envs/torch112/lib/python3.7/site-packages/torch/nn/functional.py:2 │
│ 439 in batch_norm                                                                                │
│                                                                                                  │
│   2436 │   │   _verify_batch_size(input.size())                                                  │
│   2437 │                                                                                         │
│   2438 │   return torch.batch_norm(                                                              │
│ ❱ 2439 │   │   input, weight, bias, running_mean, running_var, training, momentum, eps, torch.b  │
│   2440 │   )                                                                                     │
│   2441                                                     